# Second Checkpoint
### Predicting Pneumonia from X-Ray image

Jimena Salinas Valdespino, Santiago Segovia Baquero, Stephania Tello Zamudio, Ivanna Rodríguez Lobo

In [1]:
import os
import pandas as pd

import torch
import torch.nn as nn # basic building block for neural neteorks
import torch.nn.functional as F # import convolution functions like Relu
import torch.optim as optim # optimzer

from torch.utils import data
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as T
from torchvision.io import read_image

from sklearn import metrics

## Creating a Pytorch Dataset


In [87]:
class CustomImageDataset(Dataset):
    def __init__(self, csv_file, img_dir_path, resize=False, transform=None):
        """
        Inputs:
            - csv_file (str): file path to the csv file
            - img_dir_path: directory path to the images
            - transform: Compose (a PyTorch Class) that strings together several
              transform functions (e.g. data augmentation steps)
        """
        self.img_labels = pd.read_csv(csv_file, skiprows=1, header=None)
        self.img_dir = img_dir_path
        self.transform = transform
        self.resize = resize
        self.dimensions = self.get_dimensions()

    def __len__(self):
        """
        Returns: (int) length of your dataset
        """
        return len(self.img_labels)

    def __getitem__(self, idx):
        """
        Loads and returns your sample (the image and the label) at the
        specified index

        Parameter: idx (int): index of interest

        Returns: image, label
        """
        img_path =  os.path.join(self.img_dir, self.img_labels.iloc[idx, 0])
        
        # read the image
        image = read_image(img_path)

        # get the label
        label = self.img_labels.iloc[idx, 1]

        # apply transformations to image
        if self.transform:
            image = self.transform(image)
            
        # resize
        if self.resize:
            image = self.resize_image(image)
        
        if image.shape[0] > 1: # if it has more than one channels
            g = T.Grayscale(1)
            image = g(image)

        return image, label
    
    def get_dimensions(self):
        """
        This method creates a dictionary with the unique combinations of heightxwidth
        for each image in the dataset.

        returns a dictionary with dimensions as keys and the number of images
            with that dimension as values
        """
        dimensions = {}
        for index in range(len(self.img_labels)):
            image = self[index][0]
            if self.resize:
                image = self.resize_image(image)
            _, height, width = image.shape
            dimensions[(height,width)] = dimensions.get((height,width),0) + 1

        return dimensions
    
    def resize_image(self,image):
        """
        If the resize parameter==True, then all the images are
        converted to a 150x150 size.

        returns the resized image
        """
        transform = T.Resize((150,150))
        
        return transform(image)

Once we implemented the Dataset class, we create one object per Dataset:

There are 16 unique height x width combinations in our validation data set.

Given that there are different sizes, and some very large images, we want to standarize the size of all images. We do this by passing a `resize` boolean parameter to our `CustomImageDataset` class:

As a check, we can call on the `dictionary` attribute for each one of our datasets:

In [11]:
train_data.dimensions

{(150, 150): 5216}

In [12]:
val_data.dimensions

{(150, 150): 16}

In [13]:
test_data.dimensions

{(150, 150): 624}

## Image Augmentation

In order to avoid overfitting, we need to do image augmentation for our training
dataset. We do this below. We decided to augment the images following some 
examples of people who worked with this dataset in Kaggle.

The transformations we used are the following: rotate the image by 30 degrees, zoom into the image by 20%, flip the image horizontally, increase the image's sharpness, and change the color depth of the image.

In [85]:
train_transform = T.Compose([
    T.RandomRotation(30),
    T.RandomResizedCrop(size=(150, 150), scale=(0.8, 1.2)),
    T.RandomHorizontalFlip(),
    T.RandomAdjustSharpness(sharpness_factor=2),
    T.RandomPosterize(bits=4),
])

We then apply the above transformations to our training dataset below.

In [89]:
# Apply the transforms to the training and validation dataset
train_data = CustomImageDataset(csv_file = 'data_train.csv',
                                img_dir_path = './',
                                resize=True,
                                transform=train_transform)

In [90]:
val_data = CustomImageDataset(csv_file = 'data_val.csv',
                                img_dir_path = './',
                                resize=True,
                                transform=train_transform)

In [92]:
test_data = CustomImageDataset(csv_file = 'data_test.csv',
                                img_dir_path = './',
                                resize=True,
                                transform=train_transform)

## Creating our DataLoader

Below, we create our DataLoader. The purpose of doing this is to load our data in
batches to fit and test our model.

In [93]:
train_dataloader = DataLoader(train_data, 
                              batch_size=4, 
                              shuffle=True)

val_dataloader = DataLoader(val_data, 
                            batch_size=4, 
                            shuffle=True)

test_dataloader = DataLoader(test_data, 
                             batch_size=4, 
                             shuffle=True)

# Model

In [52]:
class CustomNeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
      
      # inspire by Turing award winning LeCun, Bengio and Hinton's paper from 1998
      # https://ieeexplore.ieee.org/document/726791 (cited more than 25,000 times!!!!!!!!!)
      # code from https://blog.paperspace.com/writing-lenet5-from-scratch-in-python/ 
      
        # self.LeNet = nn.Sequential(     
        # convolutional layers
        
        self.Layer1 = nn.Sequential(                                            # FIRST LAYER: (INPUT LAYER)
            nn.Conv2d(1, 6, kernel_size=5, stride=1, padding=0),    # CONVOLUTION 
            nn.BatchNorm2d(6),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size = 2, stride = 2))             # POOLING
        self.Layer2 = nn.Sequential(                                            # SECOND LAYER: HIDDEN LAYER 1
            nn.Conv2d(6, 16, kernel_size=5, stride=1, padding=0),   # CONVOLUTION 
            nn.BatchNorm2d(16),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size = 2, stride = 2))             # POOLING
          # fully connected layers
        self.F = nn.Flatten()
        
        self.LeNet = nn.Sequential(
            nn.Sequential(                                            
                nn.Conv2d(1, 6, kernel_size=5, stride=1, padding=0),    
                nn.BatchNorm2d(6),
                nn.ReLU(),
                nn.MaxPool2d(kernel_size = 2, stride = 2)),
            nn.Sequential(                                            
            nn.Conv2d(6, 16, kernel_size=5, stride=1, padding=0), 
            nn.BatchNorm2d(16),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size = 2, stride = 2)),
            nn.Flatten()
        )
      

      # Calculate the input size for the linear layer
        output_shape = self._get_conv_output_shape()
        print(output_shape)
        input_size = output_shape[0] * output_shape[1]

      #self.classifier = nn.Sequential(
        self.lin1 = nn.Linear(input_size, 64)
        self.relu1 = nn.ReLU()       
        self.lin2 = nn.Linear(64, 64)
        self.relu2 = nn.ReLU()              
        self.output = nn.Linear(64,2) 
                                                 
    
    def _get_conv_output_shape(self):
      # Create a dummy tensor and pass it through the convolutional layers
        x = torch.zeros((1, 1, 150, 150))
        conv_output = self.LeNet(x)
        return conv_output.shape

    def forward(self, x):
        out = self.Layer1(x)
        print("dim after first conv layer:", out.shape)
        out = self.Layer2(out)
        print("dim after second conv layer:", out.shape)
        out = self.F(out)
        print("dim after flattening:", out.shape)
        out = self.lin1(out)
        print("dim after first linear layer:", out.shape)
        out = self.relu1(out)
        print("dim after first relu:", out.shape)
        out = self.lin2(out)
        print("dim after second linear layer:", out.shape)
        out = self.relu2(out)
        print("dim after first relu:", out.shape)
        out = self.output(out)
        print("dim after output layer:", out.shape)
         
    
        return out

In [49]:
x = torch.zeros((1, 1, 150, 150))
x

tensor([[[[0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          ...,
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.]]]])

# Model's performance

In [53]:
model = CustomNeuralNetwork()

torch.Size([1, 18496])


To assess the prediction capacity of our model we compare the model's loss an classification power. The first one is defined as the accuracy of the model in terms of the predicted probabilities, while the second refers to how good the model classifies the actual labels (*e.g.*, normal lungs or lungs with pneumonia).

The loss functions that we will use are:

1. Binary cross-entropy:
2. Cross-entropy:

On the other hand, the accuracy measures we'll use are:

1. Accuracy:
2. Recall:
3. Precision:
4. F1:

**Falta dar una pequenia explicacion de estas medidas**

In [67]:
batch = next(iter(train_dataloader))

In [68]:
x, labels = batch
x.shape

torch.Size([1, 1, 150, 150])

In [69]:
x = x.float()
model(x)

dim after first conv layer: torch.Size([1, 6, 73, 73])
dim after second conv layer: torch.Size([1, 16, 34, 34])
dim after flattening: torch.Size([1, 18496])
dim after first linear layer: torch.Size([1, 64])
dim after first relu: torch.Size([1, 64])
dim after second linear layer: torch.Size([1, 64])
dim after first relu: torch.Size([1, 64])
dim after output layer: torch.Size([1, 2])


tensor([[0.0306, 0.0798]], grad_fn=<AddmmBackward0>)

In [12]:
# Define a loss function and optimizer:
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

In [94]:
# Train and validate the network
EPOCHS = 1

train_losses = []
train_accuracies = []
val_losses = []
val_accuracies = []

for _ in range(EPOCHS):  # loop over the dataset multiple times
    # TRAIN
    # Make sure gradient tracking is on, and do a pass over the data
    model.train()
    running_loss_train = 0.0
    accuracies_train = []
    f1_scores_train = []
    for i, data in enumerate(train_dataloader):
        # get the inputs; data is a list of [inputs, labels]
            inputs, labels = data
            
            inputs = inputs.float()

            # zero the parameter gradients
            optimizer.zero_grad()

            # forward + backward + optimize
            print("input shape:", inputs.shape)
            outputs = model(inputs)
            _, predicted = torch.max(outputs, 1)
            print(outputs.shape)

            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            # keep track of the loss
            running_loss_train += loss.item()
            
            # ALSO CALCULATE YOUR ACCURACY METRIC
            predicted = predicted.detach().numpy()
            labels = labels.detach().numpy()

            accuracy = metrics.accuracy_score(labels, predicted)
            accuracies_train.append(accuracy)

            f1score = metrics.f1_score(labels, predicted)
            f1_scores_train.append(f1score)

    #AVERAGE TRAINING LOSS  
    avg_train_loss = running_loss_train / (i + 1)     # i + 1 gives us the total number of batches in train dataloader
    # CALCULATE AVERAGE ACCURACY METRIC
    avg_train_acc = sum(accuracies_train)/len(accuracies_train)
    train_losses.append(avg_train_loss)
    train_accuracies.append(avg_train_acc)

    #VALIDATE
    # in the validation part, we don't want to keep track of the gradients 
    model.eval()
    running_loss_val = 0.0
    accuracies_val = []
    f1_scores_val = []
    for i, data in enumerate(val_dataloader):
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data
        
        inputs = inputs.float()

        # val prediction
        outputs = model(inputs)
        _, predicted = torch.max(outputs, 1)

        loss = criterion(outputs, labels)
        # keep track of the loss
        running_loss_val += loss.item()

        # ALSO CALCULATE YOUR ACCURACY METRIC
        predicted = predicted.detach().numpy()
        labels = labels.detach().numpy()

        accuracy = metrics.accuracy_score(labels, predicted)
        accuracies_val.append(accuracy)

        f1score = metrics.f1_score(labels, predicted)
        f1_scores_val.append(f1score)

    # AVERAGE VALIDATION LOSS
    avg_val_loss = running_loss_val / (i + 1)     # i + 1 gives us the total number of batches in train dataloader
    # CALCULATE AVERAGE ACCURACY METRIC
    avg_val_acc = sum(accuracies_val)/len(accuracies_val)
    val_losses.append(avg_val_loss)
    val_accuracies.append(avg_val_acc)            
    

input shape: torch.Size([4, 1, 150, 150])
dim after first conv layer: torch.Size([4, 6, 73, 73])
dim after second conv layer: torch.Size([4, 16, 34, 34])
dim after flattening: torch.Size([4, 18496])
dim after first linear layer: torch.Size([4, 64])
dim after first relu: torch.Size([4, 64])
dim after second linear layer: torch.Size([4, 64])
dim after first relu: torch.Size([4, 64])
dim after output layer: torch.Size([4, 2])
torch.Size([4, 2])
input shape: torch.Size([4, 1, 150, 150])
dim after first conv layer: torch.Size([4, 6, 73, 73])
dim after second conv layer: torch.Size([4, 16, 34, 34])
dim after flattening: torch.Size([4, 18496])
dim after first linear layer: torch.Size([4, 64])
dim after first relu: torch.Size([4, 64])
dim after second linear layer: torch.Size([4, 64])
dim after first relu: torch.Size([4, 64])
dim after output layer: torch.Size([4, 2])
torch.Size([4, 2])
input shape: torch.Size([4, 1, 150, 150])
dim after first conv layer: torch.Size([4, 6, 73, 73])
dim after s

input shape: torch.Size([4, 1, 150, 150])
dim after first conv layer: torch.Size([4, 6, 73, 73])
dim after second conv layer: torch.Size([4, 16, 34, 34])
dim after flattening: torch.Size([4, 18496])
dim after first linear layer: torch.Size([4, 64])
dim after first relu: torch.Size([4, 64])
dim after second linear layer: torch.Size([4, 64])
dim after first relu: torch.Size([4, 64])
dim after output layer: torch.Size([4, 2])
torch.Size([4, 2])
input shape: torch.Size([4, 1, 150, 150])
dim after first conv layer: torch.Size([4, 6, 73, 73])
dim after second conv layer: torch.Size([4, 16, 34, 34])
dim after flattening: torch.Size([4, 18496])
dim after first linear layer: torch.Size([4, 64])
dim after first relu: torch.Size([4, 64])
dim after second linear layer: torch.Size([4, 64])
dim after first relu: torch.Size([4, 64])
dim after output layer: torch.Size([4, 2])
torch.Size([4, 2])
input shape: torch.Size([4, 1, 150, 150])
dim after first conv layer: torch.Size([4, 6, 73, 73])
dim after s

input shape: torch.Size([4, 1, 150, 150])
dim after first conv layer: torch.Size([4, 6, 73, 73])
dim after second conv layer: torch.Size([4, 16, 34, 34])
dim after flattening: torch.Size([4, 18496])
dim after first linear layer: torch.Size([4, 64])
dim after first relu: torch.Size([4, 64])
dim after second linear layer: torch.Size([4, 64])
dim after first relu: torch.Size([4, 64])
dim after output layer: torch.Size([4, 2])
torch.Size([4, 2])
input shape: torch.Size([4, 1, 150, 150])
dim after first conv layer: torch.Size([4, 6, 73, 73])
dim after second conv layer: torch.Size([4, 16, 34, 34])
dim after flattening: torch.Size([4, 18496])
dim after first linear layer: torch.Size([4, 64])
dim after first relu: torch.Size([4, 64])
dim after second linear layer: torch.Size([4, 64])
dim after first relu: torch.Size([4, 64])
dim after output layer: torch.Size([4, 2])
torch.Size([4, 2])
input shape: torch.Size([4, 1, 150, 150])
dim after first conv layer: torch.Size([4, 6, 73, 73])
dim after s

input shape: torch.Size([4, 1, 150, 150])
dim after first conv layer: torch.Size([4, 6, 73, 73])
dim after second conv layer: torch.Size([4, 16, 34, 34])
dim after flattening: torch.Size([4, 18496])
dim after first linear layer: torch.Size([4, 64])
dim after first relu: torch.Size([4, 64])
dim after second linear layer: torch.Size([4, 64])
dim after first relu: torch.Size([4, 64])
dim after output layer: torch.Size([4, 2])
torch.Size([4, 2])
input shape: torch.Size([4, 1, 150, 150])
dim after first conv layer: torch.Size([4, 6, 73, 73])
dim after second conv layer: torch.Size([4, 16, 34, 34])
dim after flattening: torch.Size([4, 18496])
dim after first linear layer: torch.Size([4, 64])
dim after first relu: torch.Size([4, 64])
dim after second linear layer: torch.Size([4, 64])
dim after first relu: torch.Size([4, 64])
dim after output layer: torch.Size([4, 2])
torch.Size([4, 2])
input shape: torch.Size([4, 1, 150, 150])
dim after first conv layer: torch.Size([4, 6, 73, 73])
dim after s

input shape: torch.Size([4, 1, 150, 150])
dim after first conv layer: torch.Size([4, 6, 73, 73])
dim after second conv layer: torch.Size([4, 16, 34, 34])
dim after flattening: torch.Size([4, 18496])
dim after first linear layer: torch.Size([4, 64])
dim after first relu: torch.Size([4, 64])
dim after second linear layer: torch.Size([4, 64])
dim after first relu: torch.Size([4, 64])
dim after output layer: torch.Size([4, 2])
torch.Size([4, 2])
input shape: torch.Size([4, 1, 150, 150])
dim after first conv layer: torch.Size([4, 6, 73, 73])
dim after second conv layer: torch.Size([4, 16, 34, 34])
dim after flattening: torch.Size([4, 18496])
dim after first linear layer: torch.Size([4, 64])
dim after first relu: torch.Size([4, 64])
dim after second linear layer: torch.Size([4, 64])
dim after first relu: torch.Size([4, 64])
dim after output layer: torch.Size([4, 2])
torch.Size([4, 2])
input shape: torch.Size([4, 1, 150, 150])
dim after first conv layer: torch.Size([4, 6, 73, 73])
dim after s

input shape: torch.Size([4, 1, 150, 150])
dim after first conv layer: torch.Size([4, 6, 73, 73])
dim after second conv layer: torch.Size([4, 16, 34, 34])
dim after flattening: torch.Size([4, 18496])
dim after first linear layer: torch.Size([4, 64])
dim after first relu: torch.Size([4, 64])
dim after second linear layer: torch.Size([4, 64])
dim after first relu: torch.Size([4, 64])
dim after output layer: torch.Size([4, 2])
torch.Size([4, 2])
input shape: torch.Size([4, 1, 150, 150])
dim after first conv layer: torch.Size([4, 6, 73, 73])
dim after second conv layer: torch.Size([4, 16, 34, 34])
dim after flattening: torch.Size([4, 18496])
dim after first linear layer: torch.Size([4, 64])
dim after first relu: torch.Size([4, 64])
dim after second linear layer: torch.Size([4, 64])
dim after first relu: torch.Size([4, 64])
dim after output layer: torch.Size([4, 2])
torch.Size([4, 2])
input shape: torch.Size([4, 1, 150, 150])
dim after first conv layer: torch.Size([4, 6, 73, 73])
dim after s

/Users/ianjeong/opt/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1580: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, "true nor predicted", "F-score is", len(true_sum))


input shape: torch.Size([4, 1, 150, 150])
dim after first conv layer: torch.Size([4, 6, 73, 73])
dim after second conv layer: torch.Size([4, 16, 34, 34])
dim after flattening: torch.Size([4, 18496])
dim after first linear layer: torch.Size([4, 64])
dim after first relu: torch.Size([4, 64])
dim after second linear layer: torch.Size([4, 64])
dim after first relu: torch.Size([4, 64])
dim after output layer: torch.Size([4, 2])
torch.Size([4, 2])
input shape: torch.Size([4, 1, 150, 150])
dim after first conv layer: torch.Size([4, 6, 73, 73])
dim after second conv layer: torch.Size([4, 16, 34, 34])
dim after flattening: torch.Size([4, 18496])
dim after first linear layer: torch.Size([4, 64])
dim after first relu: torch.Size([4, 64])
dim after second linear layer: torch.Size([4, 64])
dim after first relu: torch.Size([4, 64])
dim after output layer: torch.Size([4, 2])
torch.Size([4, 2])
input shape: torch.Size([4, 1, 150, 150])
dim after first conv layer: torch.Size([4, 6, 73, 73])
dim after s

input shape: torch.Size([4, 1, 150, 150])
dim after first conv layer: torch.Size([4, 6, 73, 73])
dim after second conv layer: torch.Size([4, 16, 34, 34])
dim after flattening: torch.Size([4, 18496])
dim after first linear layer: torch.Size([4, 64])
dim after first relu: torch.Size([4, 64])
dim after second linear layer: torch.Size([4, 64])
dim after first relu: torch.Size([4, 64])
dim after output layer: torch.Size([4, 2])
torch.Size([4, 2])


KeyboardInterrupt: 

In [57]:
train_losses

[0.42117161921611646]

In [58]:
train_accuracies

[0.8024009146341463]

In [59]:
val_losses

[1.1948494911193848]

In [60]:
val_accuracies

[0.5625]

In [95]:
# Testing the model

model.eval()
running_loss_test = 0.0
accuracies_test = []
f1_scores_test = []
for i, data in enumerate(test_dataloader):
    # get the inputs; data is a list of [inputs, labels]
    inputs, labels = data
    inputs = inputs.float()

    # test prediction
    outputs = model(inputs)
    _, predicted = torch.max(outputs, 1)

    # ALSO CALCULATE YOUR ACCURACY METRIC
    predicted = predicted.detach().numpy()
    labels = labels.detach().numpy()

    accuracy = metrics.accuracy_score(labels, predicted)
    accuracies_test.append(accuracy)

    f1score = metrics.f1_score(labels, predicted)
    f1_scores_test.append(f1score)

# AVERAGE VALIDATION LOSS
avg_test_loss = running_loss_test / (i + 1)     # i + 1 gives us the total number of batches in train dataloader
# CALCULATE AVERAGE ACCURACY METRIC
avg_test_acc = sum(accuracies_test)/len(accuracies_test)   
    
    

dim after first conv layer: torch.Size([4, 6, 73, 73])
dim after second conv layer: torch.Size([4, 16, 34, 34])
dim after flattening: torch.Size([4, 18496])
dim after first linear layer: torch.Size([4, 64])
dim after first relu: torch.Size([4, 64])
dim after second linear layer: torch.Size([4, 64])
dim after first relu: torch.Size([4, 64])
dim after output layer: torch.Size([4, 2])
dim after first conv layer: torch.Size([4, 6, 73, 73])
dim after second conv layer: torch.Size([4, 16, 34, 34])
dim after flattening: torch.Size([4, 18496])
dim after first linear layer: torch.Size([4, 64])
dim after first relu: torch.Size([4, 64])
dim after second linear layer: torch.Size([4, 64])
dim after first relu: torch.Size([4, 64])
dim after output layer: torch.Size([4, 2])
dim after first conv layer: torch.Size([4, 6, 73, 73])
dim after second conv layer: torch.Size([4, 16, 34, 34])
dim after flattening: torch.Size([4, 18496])
dim after first linear layer: torch.Size([4, 64])
dim after first relu: t

KeyboardInterrupt: 